## First Step: create viewsheds based on 6 image locations
1. Load Data
    * DEMS
    * GPS csv file
2. Convert csv file to coordinates
3. Convert GPS coordinates (a feature layer) to shp
4. Create individual shps for each image point
    * Extract relevant values for names (names for new shapefiles)
    * Create 6 copies of the GPS shp (containing all data) and delete all rows expt 1 point (then save shp)
5. Use Viewshed() to calculate viewshed rasters for each image


<i>Please Note this Notebook was completed within an ArcGIS Pro environment with access to Arcpy</i>

In [4]:
# Import system modules
import arcpy
from arcpy import env
from arcpy.sa import *
from pathlib import Path

In [7]:
########################################## Creating Variables for Project ##################################################
# adjust to accurate location of files
GPS_csv_Filepath = r"C:\Users\runac\Downloads\Fall_2022\Programming\project_idea\JupNotebooks\GPS_metadata.csv"
DEM_Filepath = r"D:\Fall_2022\Programming\Project\Data\30DEM\USGS_1_n49w114_20210607.tif"
Output_Filepath = "D:\Fall_2022\Programming\Project\Viewshed_MT_cleaner\Viewshed_MT_cleaner.gdb"
#saving files to a NON .gdb directory
out_path = r"D:\Fall_2022\Programming\Project\Viewshed_MT_cleaner"
Num_Image_Coords = 6

### Declaring functions used throughout this file. 

In [8]:
def Creating_file_names(path, filename_str, length):
    """
    Provide the path where you'd like to save each file, and the naming string used to name resulting files
    filename_str needs to be formatted as "\<filename>", and the supplied path must use '\' 
    
    Returns a sequential numbered list of filename paths for the provided length 
    (length = number of image coordinates being used to create viewsheds)
    """
    path_ = path
    list_ = []
    for number in range(length): 
        filename = filename_str + str(number + 1)
        list_.append(path_ + filename)
    
    return list_

#################### Connect image.shp point to each of the points on circle perimeter creating a new polyline shapefile #########
# create a def to be called later that extracts the coordinates for each image.shp
def get_image_coords(path_to_image):
    '''
    Given the shapefile path you can extract the points coorndinates. 
    Note: this function assumes provided shapefile is 1 point, thus it only returns 1 point value, 
    if shapefile has many pts, funtion will return last pt
    
    Returns an arcpy Point
    '''
    with arcpy.da.SearchCursor(path_to_image, ['*']) as cursor:
        for row in cursor:
            cp = arcpy.Point(row[1][0], row[1][1])
    return cp

def creating_polylines(filepath, anchor_point, connection_points, percentage_increment): 
    """
    Creates lines from each coordinate (image<#>.shp) to the points plotted on the minimum bounding circle 
    created around the viewshed. 
    
    filepath: insert desired name for each polyline file created with entire filepath (to .gdb)
    anchor_point: filepath to shapefile for each respective image's coordinates
    connection_points: filepath to the points created on the minimum bounding circle
    percentage_increment: dictates the frequency/ interval of the points plotted around the minimum bounding circle.
    The samller the number the more points. Must be a value between 0 and 100. 
    (makes sure this 'percentage_increment' = 'Percentage' in "GeneratePointsAlongLines()")
    
    Returns a new shapefile made of polylines labeled by "Degree" column
    """
    # creating new file name and shapefile for polylines
    new_fc = filepath
    name = filepath[(len(Output_Filepath)+1):len(filepath)]
    # matching the spatial refernce of the DEM file
    sr = arcpy.SpatialReference(4269) 
    arcpy.management.CreateFeatureclass(Output_Filepath, name, 'POLYLINE', spatial_reference = sr)
    # ADDing a column that designates degree (I believe arc designates N as 0 and rotates clockwise)
    arcpy.management.AddField(name, "Degree", "DOUBLE", None, None, None, "Degree", "NULLABLE", "NON_REQUIRED", '')

    percentage = 0
    
    # Creating a line for each point on perimeter to corresponding image
    # looping through perimeter points
    with arcpy.da.SearchCursor(connection_points, ['*']) as cursor:
        for row in cursor:
            list_points = arcpy.Array([])
            # calulating the degree sequentially clockwise (adjust percentage if GeneratePointsAlongLines() uses differnt percentage)
            degree = (percentage/100)*360
            
            # declaring start (image location) & end (pts on circle) points for each line
            cp1 = get_image_coords(anchor_point) #pulling the same image point
            cp2 = arcpy.Point(row[1][0], row[1][1])
            list_points.append(cp2)
            list_points.append(cp1)
            
            # assignng spatial reference (should check to make sure its accurate)
            spatial_reference = arcpy.SpatialReference(4326)
            polyline = arcpy.Polyline(list_points, spatial_reference)
           
            # inserting new line to new feature class
            with arcpy.da.InsertCursor(new_fc, ["SHAPE@", "Degree"]) as icur:
                icur.insertRow([polyline, degree])
                del icur
            
            # updating percentage for each line
            percentage += percentage_increment

In [9]:
########################################## Loading DEM and GPS data ###########################################################
# (steps 1-4)
# relevant rasters needed for the viewshed...
DEM1 = Raster(DEM_Filepath)
# pre-defining spatial refernce
# spatial_ref = arcpy.Describe(GPS_csv_Filepath).spatialReference
spatial_ref = arcpy.SpatialReference(4326)

# convert the csv table to points (run once)
arcpy.management.XYTableToPoint(GPS_csv_Filepath, 
                                r"GPS_metadata_XYTableToPoint", 
                                "decimal_Long", 
                                "decimal_Lat",
                                "GPSAltitude", 
                                spatial_ref)

# converting GPS data to a shp
arcpy.conversion.FeatureClassToShapefile("GPS_metadata_XYTableToPoint", Output_Filepath)

# creating a list that stores the name of each image file (used later to create individual shp for each point)
List_shp = []
List_names = []
with arcpy.da.UpdateCursor("GPS_metadata_XYTableToPoint", ['Field1']) as cursor:
    for row in cursor:
        print(row[0])
        name = row[0].split('.')
        List_shp.append(f'{name[0]}.shp')
        List_names.append(row[0])
        
# copying featureclass and converting to individual shpfiles for each point
data = Output_Filepath + "\GPS_metadata_XYTableToPoint"
count = 0
for name in List_shp:
    # copying original file 
    arcpy.management.CopyFeatures(data, name)
    # creating shpfiles with 1 point by deleting all features excepts for 1 row
    with arcpy.da.UpdateCursor(name, ['Field1','decimal_La','decimal_Lo', 'GPSAltit_1']) as cursor:
        for row in cursor:
            if row[0] != List_names[count]:
                cursor.deleteRow()
    count += 1

image1.HEIC
image2.HEIC
image3.HEIC
image4.JPG
image5.JPG
image8.JPG


In [10]:
############### Creating a list of names to use for viewsheds, based on the number of images processed ###################
# (step 5)
# declaring lists for names and creating files for Viewshed Outputs
List_Raster_V_names = Creating_file_names(Output_Filepath, "\outvwdshd", Num_Image_Coords)
List_outnames = Creating_file_names(Output_Filepath, "\outViewshed", Num_Image_Coords)

###################################################### VIEWSHED ############################################################
for shp, out_file, vwshd in zip(List_shp, List_outnames, List_Raster_V_names):
    # Execute Viewshed (DON'T NEED 'List_Raster_V_names')
    outViewshed = Viewshed(DEM1, shp, 1, "FLAT_EARTH", 0.13)
    # Save the output 
    outViewshed.save(out_file)
    name = vwshd[(len(Output_Filepath)+1):len(vwshd)]
    print("saving", name)


saving outvwdshd1
saving outvwdshd2
saving outvwdshd3
saving outvwdshd4
saving outvwdshd5
saving outvwdshd6


## Next big step: extracting elevation values (maxes) in a 360 view around image points. 

1. Viewshed raster to Polygon
2. Create a minimum bounding circle on vectorized viewshed raster
3. Generate points on perimeter of circle (min bounding region)
4. Connect image.shp point to each of the points on circle perimeter creating a new polyline shapefile.
5. Use vectorized viewshed raster to mask DEM (resulting in a viewshed shaped DEM with real elevation values)
6. Extract the cell values from the clipped viewshed DEM with the Polylines. 
    * For each line store the largest DEM value and save values in a table
7. Plot table (in LATIS see Visualization_elev_dist.ipynb notebook)

In [11]:
################################################## Viewshed raster to Polygon ########################################
# vectorizing resulting viewshed rasters

# list of names for the viewshed tifs
vwshd_tifs = Creating_file_names(Output_Filepath, "\outViewshed", Num_Image_Coords)
# list of names to be used for vectorized viewsheds
ras_to_polys = Creating_file_names(Output_Filepath, "\Poly_outView", Num_Image_Coords)

# Looping over "ras_to_polys" to convert ALL viewsheds to polygons
for tif, poly in zip(vwshd_tifs, ras_to_polys):
    # isolating names of tif file from rest of path
    name = tif[(len(Output_Filepath)+ 1):len(tif)]
    # converting rasters to polygons
    arcpy.conversion.RasterToPolygon(name, 
                                     poly, 
                                     "SIMPLIFY", 
                                     "Value", 
                                     "SINGLE_OUTER_PART", 
                                     None)

# deleting polygons with gridcode = 0 (represents areas viewer cannot see)
for v in ras_to_polys: 
    with arcpy.da.UpdateCursor(v, ['*']) as cursor:
        for row in cursor:
            if row[3] == 0:
                cursor.deleteRow()
    print("Saving & Updating:", v[(len(Output_Filepath)+1):len(v)])

Saving & Updating: Poly_outView1
Saving & Updating: Poly_outView2
Saving & Updating: Poly_outView3
Saving & Updating: Poly_outView4
Saving & Updating: Poly_outView5
Saving & Updating: Poly_outView6


In [12]:
########################### create a min bounding circle on vectorized viewshed raster #######################
# creating list of names for minimum bounding circles
min_bound_names = Creating_file_names(Output_Filepath, "\B_outView", Num_Image_Coords)

# creating minimum bounding circle around viewshds
for poly_vwshd, min_bound in zip(ras_to_polys, min_bound_names):
    arcpy.management.MinimumBoundingGeometry(poly_vwshd, 
                                         min_bound, 
                                         "CIRCLE", 
                                         "ALL")
    print("Saving", min_bound[(len(Output_Filepath)+1):len(min_bound)])

Saving B_outView1
Saving B_outView2
Saving B_outView3
Saving B_outView4
Saving B_outView5
Saving B_outView6


In [13]:
############################## Generate points on perimeter of circle (min bounding region) ######################
# creating list of names for points on minimum bouding circles
pts_names = Creating_file_names(Output_Filepath, "\Pts_View", Num_Image_Coords)

# creating pts on minimum bounding circle
for circle, pts in zip(min_bound_names, pts_names):
    arcpy.management.GeneratePointsAlongLines(circle, 
                                          pts, 
                                          "PERCENTAGE",  
                                          Percentage=0.2, 
                                          Include_End_Points='END_POINTS')
    
    print("Saving", pts[(len(Output_Filepath)+1):len(pts)])

    # takes apporx 2-3 mins to execute

Saving Pts_View1
Saving Pts_View2
Saving Pts_View3
Saving Pts_View4
Saving Pts_View5
Saving Pts_View6


In [14]:
#################### Connect image.shp point to each of the points on circle perimeter creating a new polyline shapefile ##########

# creating paths and names for new polyline files 
polyline_names = Creating_file_names(Output_Filepath, "\View_lines", Num_Image_Coords)

# polyline_names : recently declared above
# pts_names : calling this already made list of minimum bounding circle pts
# List_shp : calling this already made list of image locations 

# Running the function that'll create polylines for each image and corresponding viewshed points
for polyline, image, pts, in zip(polyline_names, List_shp, pts_names):
    print("Creating:", polyline[(len(Output_Filepath) + 1):len(polyline)])
    creating_polylines(polyline, image, pts, 0.2)
    
# takes 1.5 mins to make lines when p = 0.2

Creating: View_lines1
Creating: View_lines2
Creating: View_lines3
Creating: View_lines4
Creating: View_lines5
Creating: View_lines6


In [15]:
########################################## Use vectorized viewshed raster to mask DEM ###################################
# creating paths and names for new clipped DEM of viewsheds 
c_DEM_names = Creating_file_names(Output_Filepath, "\Clipped_DEM_Vwshd", Num_Image_Coords)

# declaring the spatial extent of the DEM1 (used to help clip the DEM to the viewshed output)
extent = DEM1.extent
xmin = extent.XMin
ymin = extent.YMin
xmax = extent.XMax
ymax = extent.YMax
spatial_extent = str(xmin) + " " + str(ymin) +  " " + str(xmax) +  " " +str(ymax)

# creating a clipped extent of DEM for each image
for vwshd, clip_dem in zip(ras_to_polys, c_DEM_names):
    print("Clipping:", clip_dem[(len(Output_Filepath)+1):len(clip_dem)])
    poly_vwshd = vwshd[(len(Output_Filepath)+1):len(vwshd)]
    arcpy.management.Clip(DEM1, 
                          spatial_extent,
                          clip_dem, 
                          poly_vwshd, 
                          "-999999", 
                          "ClippingGeometry", 
                          "NO_MAINTAIN_EXTENT")

Clipping: Clipped_DEM_Vwshd1
Clipping: Clipped_DEM_Vwshd2
Clipping: Clipped_DEM_Vwshd3
Clipping: Clipped_DEM_Vwshd4
Clipping: Clipped_DEM_Vwshd5
Clipping: Clipped_DEM_Vwshd6


In [16]:
############### Extract the Elevation values from the clipped viewshed DEM with the Polylines ############
# creating paths and names for new INTIGER-(datatype) clipped DEM of viewsheds 
int_DEM_names = Creating_file_names(Output_Filepath, "\Int_clip_DEM_Vwshd", Num_Image_Coords)
poly_pixels_vw = Creating_file_names(Output_Filepath, "\Poly_Int_Clip_DEM", Num_Image_Coords)
max_elevation = Creating_file_names(Output_Filepath, "\lines_max_elev", Num_Image_Coords)

# convert elevation datatype from decimal to integer (looses some data)
for clip_dem, int_dem in zip(c_DEM_names, int_DEM_names):
    print("Saving", int_dem[(len(Output_Filepath)+1):len(int_dem)])
    out_raster = arcpy.ia.Int(clip_dem[(len(Output_Filepath)+1):len(clip_dem)]); out_raster.save(int_dem)

# convert raster to polygons (s.t. each raster pixel is a polygon)
for int_dem, poly_dem in zip(int_DEM_names, poly_pixels_vw):
    arcpy.conversion.RasterToPolygon(int_dem[(len(Output_Filepath)+1):len(int_dem)], 
                                     poly_dem, 
                                     "NO_SIMPLIFY", 
                                     "Value", 
                                     "SINGLE_OUTER_PART", 
                                     None)

# spatial join between pixels (as polygons) and polylines : whilst saving only max elevations for each polyline
# this join returns the max 'gridcode' ('elevation value') that intersects with a line!!
for lines, poly_dem, elev in zip(polyline_names, poly_pixels_vw, max_elevation):
    arcpy.analysis.SpatialJoin(lines[(len(Output_Filepath)+1):len(lines)], 
                               poly_dem[(len(Output_Filepath)+1):len(poly_dem)], 
                               elev, 
                               "JOIN_ONE_TO_ONE", 
                               "KEEP_ALL", 
                               'Degree "Degree" true true false 19 Double 0 0,First,#,view1_lines_b,Degree,-1,-1;'\
                               'gridcode "gridcode" true true false 4 Long 0 0,Max,#,Poly_Int_Clip_DEM1,gridcode,-1,-1', 
                               "INTERSECT", 
                               None, 
                               '')
    print("Saving Max Elevation value")

Saving Int_clip_DEM_Vwshd1
Saving Int_clip_DEM_Vwshd2
Saving Int_clip_DEM_Vwshd3
Saving Int_clip_DEM_Vwshd4
Saving Int_clip_DEM_Vwshd5
Saving Int_clip_DEM_Vwshd6
Saving Max Elevation value
Saving Max Elevation value
Saving Max Elevation value
Saving Max Elevation value
Saving Max Elevation value
Saving Max Elevation value


In [17]:
# exported the shp as a csv to view in LATIS
# exporting csv files to a NON gdb directoy
csv_names = Creating_file_names(out_path, "\lines_max_elev", Num_Image_Coords)

for elev, csv in zip(max_elevation, csv_names):
    name = csv + ".csv"
    print("Saving:", name[(len(out_path)+1):(len(csv)+4)])
    arcpy.conversion.ExportTable(elev[(len(Output_Filepath)+1):len(elev)], 
                                 name, 
                                 '', 
                                 "NOT_USE_ALIAS", 
                                 'Join_Count "Join_Count" true true false 4 Long 0 0,First,#,view1_lines_b_SpatialJoin2,Join_Count,-1,-1;'\
                                 'TARGET_FID "TARGET_FID" true true false 4 Long 0 0,First,#,view1_lines_b_SpatialJoin2,TARGET_FID,-1,-1;'\
                                 'Degree "Degree" true true false 8 Double 0 0,First,#,view1_lines_b_SpatialJoin2,Degree,-1,-1;'\
                                 'gridcode "gridcode" true true false 4 Long 0 0,First,#,view1_lines_b_SpatialJoin2,gridcode,-1,-1', 
                                 None)

Saving: lines_max_elev1.csv
Saving: lines_max_elev2.csv
Saving: lines_max_elev3.csv
Saving: lines_max_elev4.csv
Saving: lines_max_elev5.csv
Saving: lines_max_elev6.csv


##### After successfully extrating the Max elevation values for each polyline, I need to measure distance from observer/image pt to location of Max elevation pixel. I then need to create new values where max elevation is divided by distance (to accurately depict a mountain ridgeline w.r.t. distance) 



### Another Big Step: Calculate Distance from image/anchor pt to each Max Elevation 

1. Spatial Join between vectorized clipped DEM from viewshed and the polylines generated from image to min bounding pts
    * retain polygon (pixel) geometry
    * make sure it retains only MAX elevation polygon per line
2. Clip or divide the Polylines with the geom returned from previous step. 
    * retain only lines that intersect with image/anchor point
3. Measure the line length (Use "Calculate Geometry Attributes" tool)

#### Defining more functions used in the following code:

In [21]:
############################# Only working way to indentify MAX for each polyline... #######################################
# identifying the max elevation for each polyline and saving the polygon OBJECTID, coords, and elevation to a dictionary

def Identifying_Max_elevations_from_Poly_Pixels(line_geom, poly_pixel_geom):
    """
    Returns a dictionary that identifes the vectorized pixels that contain the MAX elevation for each polyline. 
    dictionary format: max_dict[Degree] = [pixel_ID, coords (lat,long), Max_Elevation]
    Assumes that the "poly_pixel_geom" feature always ahs the same columns in a certain order (will extract worng values 
    if columns are not the same for all geoms)
    """
    max_dict = {}
    counter = 0
    # For each line:
    with arcpy.da.SearchCursor(line_geom, ['Degree']) as cursor:
        for line in cursor:
            ID = line[0] # make sure to grab "Degree" so as to match unique values later
        # iterate over corresponding pixels that match each line (through 'ID' column)
            with arcpy.da.SearchCursor(poly_pixel_geom, ['*']) as cur:
                Max_ = 0
                obj_id = 0
                coords = 0
                for pixel in cur:
                    counter += 1
                    # make sure these indeces match correct columns!!
                    if pixel[6] == ID:
                        if Max_ < pixel[5]:
                            Max_ = pixel[5]
                            obj_id = pixel[3] #target_ID
                            coords = pixel[1]
                        max_dict[ID] = [obj_id, coords, Max_]
    print("Done Creating Max Dict. Iterated", counter, "times (lines x pixels in each viewshed)")                
    return max_dict
# Fucntion works (with print functions) BUT I bet there is a better/faster way...

def Keeping_poly_pixels_with_Max_Elevation_for_each_Polyline(max_dict, poly_pixel_geom):
    """
    This function deletes the poylgons from the geometry vectorized pixels that have not been identified as the 
    pixels with max elevation for each polyline. 
    Returns a subset/edited version of the provided "poly_pixel_geom" (will only work if the proper inputs have already been made)
    """
    # grabbing the OBJECTID's and using these values to delete non-max associated polygons
    Obj_ID_list = []
    for item in max_dict.values():
        # there are pixel polygons IDs that are duplicated in this list, only need to add them once (no need for repeating IDs)
        if item[0] not in Obj_ID_list:
            Obj_ID_list.append(item[0])

    safety = 0
    # deleteting polygons whose target_IDs are NOT the max_dict max_dict) 
    with arcpy.da.UpdateCursor(poly_pixel_geom, ['*']) as cursor:
        for row in cursor:
            if row[3] not in Obj_ID_list and safety < 4:
    #             print(row[3])
    #             safety +=1
                cursor.deleteRow()

    # still need to delete more pixel polygons if there do not conatin the max elevetion pre line...(slower than the above Search Cursor)
    with arcpy.da.UpdateCursor(poly_pixel_geom, ['*']) as cursor:
        for row in cursor:
            # checking if the Target Id and the elevation are within the max_dict, if not delete the row
            if row[3] not in max_dict[row[6]] and row[5] not in max_dict[row[6]]:
                cursor.deleteRow()
                
######################################### Clip lines where they intersect MAX pixels ##########################################
def Splitting_polylines_by_Max_Pixel_geom(poly_pixel_max_geom, intersection_geom, polylines_geom, Split_line_geom, max_dict):
    # rename "Target_FID" col inside the vectorixed pixel geom features that have been spatialy joined with lines...
    # make a new field then copy the old field values into the new one 
    arcpy.management.CalculateField(poly_pixel_max_geom, "PIXEL_ID", "!TARGET_FID!", "PYTHON3", '', "LONG", "NO_ENFORCE_DOMAINS")
    # delete unnecessary columns
    arcpy.management.DeleteField(poly_pixel_max_geom, "TARGET_FID", "DELETE_FIELDS")

    # creating points that delinate the intersect of each line with each max polygon (creates points whenver each line intersect a polygon)
    arcpy.analysis.Intersect([polylines_geom, poly_pixel_max_geom], 
                             intersection_geom, 
                             "NO_FID", 
                             None, 
                             "POINT")

    # need to keep only one point per line that marks where each line intersects withits max elevation pixel/polygon
    safety = 0
    number_of_matches = 0
    with arcpy.da.UpdateCursor(intersection_geom, ['gridcode', 'Degree', 'PIXEL_ID']) as cursor:
        for row in cursor:
            safety += 1
            if safety < 10000:
                degree = row[1]
                #if pixel_id are equal and elevation values are equal keep
                if  max_dict[degree][0] == row[2] and max_dict[degree][2] == row[0]:
                    number_of_matches += 1
                # else delete row
                else:
                    cursor.deleteRow()

    print("Number of matches found:", number_of_matches)      


    # Split lines by points:
    arcpy.management.SplitLineAtPoint(polylines_geom, 
                                      intersection_geom, 
                                      Split_line_geom, 
                                      None)

    # delete polyline with ORG_SEG = 1 (delete lines that do not intersect with anchor point):
    safety = 0
    with arcpy.da.UpdateCursor(Split_line_geom, ['ORIG_SEQ']) as cursor:
        for row in cursor:
            if row[0] == 1:
                cursor.deleteRow()

    # Successfully clipped lines!!!

In [18]:
############################## Spatial Join between lines and DEM pixels Keeping Elevation Geom ###############################
Poly_Int_Clip_DEM_SpJn = Creating_file_names(Output_Filepath, "\Poly_Int_Clip_DEM_SpJn", Num_Image_Coords)

# creating a Spatial Join where we keep the elevation geom: 
for poly_pixel, max_el, poly_SpJn in zip(poly_pixels_vw, max_elevation, Poly_Int_Clip_DEM_SpJn):
    arcpy.analysis.SpatialJoin(poly_pixel, 
                               max_el, 
                               poly_SpJn, 
                               "JOIN_ONE_TO_MANY", 
                               "KEEP_COMMON", 
                               'gridcode "gridcode" true true false 4 Long 0 0,First,#,Poly_Int_Clip_DEM1,gridcode,-1,-1;'\
                               'Degree "Degree" true true false 19 Double 0 0,First,#,view1_lines_b,Degree,-1,-1;'\
                               'Line_ID "Line_ID" true true false 19 Double 0 0,First,#,view1_lines_b,Line_ID,-1,-1', 
                               "INTERSECT", 
                               None, 
                               '')
# this takes a WHILE (at least 5 minutes)

In [20]:
# indentifying MAX elevation for each polyline
max_dict1 = Identifying_Max_elevations_from_Poly_Pixels("lines_max_elev1", "Poly_Int_Clip_DEM_SpJn1")
max_dict2 = Identifying_Max_elevations_from_Poly_Pixels("lines_max_elev2", "Poly_Int_Clip_DEM_SpJn2")
max_dict3 = Identifying_Max_elevations_from_Poly_Pixels("lines_max_elev3", "Poly_Int_Clip_DEM_SpJn3")
max_dict4 = Identifying_Max_elevations_from_Poly_Pixels("lines_max_elev4", "Poly_Int_Clip_DEM_SpJn4")
max_dict5 = Identifying_Max_elevations_from_Poly_Pixels("lines_max_elev5", "Poly_Int_Clip_DEM_SpJn5")
max_dict6 = Identifying_Max_elevations_from_Poly_Pixels("lines_max_elev6", "Poly_Int_Clip_DEM_SpJn6")
# again, this can take a few minutes ~12 minutes

Done Creating Max Dict. Iterated 7881231 times (lines x pixels in each viewshed)
Done Creating Max Dict. Iterated 15475389 times (lines x pixels in each viewshed)
Done Creating Max Dict. Iterated 20096613 times (lines x pixels in each viewshed)
Done Creating Max Dict. Iterated 13940826 times (lines x pixels in each viewshed)
Done Creating Max Dict. Iterated 13922790 times (lines x pixels in each viewshed)
Done Creating Max Dict. Iterated 8151270 times (lines x pixels in each viewshed)


In [22]:
# deleting poylgons from the geometry pixels that have not been identified as the pixels with max elevation for each polyline
Keeping_poly_pixels_with_Max_Elevation_for_each_Polyline(max_dict1, "Poly_Int_Clip_DEM_SpJn1")
Keeping_poly_pixels_with_Max_Elevation_for_each_Polyline(max_dict2, "Poly_Int_Clip_DEM_SpJn2")
Keeping_poly_pixels_with_Max_Elevation_for_each_Polyline(max_dict3, "Poly_Int_Clip_DEM_SpJn3")
Keeping_poly_pixels_with_Max_Elevation_for_each_Polyline(max_dict4, "Poly_Int_Clip_DEM_SpJn4")
Keeping_poly_pixels_with_Max_Elevation_for_each_Polyline(max_dict5, "Poly_Int_Clip_DEM_SpJn5")
Keeping_poly_pixels_with_Max_Elevation_for_each_Polyline(max_dict6, "Poly_Int_Clip_DEM_SpJn6")

In [23]:
# creating variables names for functions
intersection_pts_names = Creating_file_names(Output_Filepath, "\lines_poly_Intersect", Num_Image_Coords)
Split_line_geom_names = Creating_file_names(Output_Filepath, "\lines_Split_view", Num_Image_Coords)
max_dicts = [max_dict1, max_dict2, max_dict3, max_dict4, max_dict5, max_dict6]

# clipping lines based on their intersection with Max pixels for each line
for pixel_geom, intersection_pts, max_lines, split_lines, dict_ in zip( Poly_Int_Clip_DEM_SpJn, intersection_pts_names, max_elevation, Split_line_geom_names, max_dicts):
    splits = pixel_geom.split('\\')
    name = splits[len(splits)-1]
    splits_line = max_lines.split('\\')
    name_line = splits_line[len(splits_line)-1]
    string  = "'" + max_lines[len(max_lines)-len(name_line):len(max_lines)] +"'" + ' #;'+ "'"+ pixel_geom[len(pixel_geom)-len(name):len(pixel_geom)]+ "'" + ' #'
    
    Splitting_polylines_by_Max_Pixel_geom(pixel_geom, intersection_pts, max_lines, split_lines, dict_)

Number of matches found: 1553
Number of matches found: 713
Number of matches found: 655
Number of matches found: 955
Number of matches found: 766
Number of matches found: 3539


In [24]:
########################################### Calculate Geometry (line Length) ###############################################
for split_lines in Split_line_geom_names:
    # Calculating Line Length for each line (geodesic) into a new column named "LENGTH"
    arcpy.management.CalculateGeometryAttributes(split_lines, 
                                                 "LENGTH LENGTH_GEODESIC", 
                                                 "KILOMETERS", '', 
                                                 'GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]', 
                                                 "SAME_AS_INPUT")

In [25]:
# exporting Distances associated with each of the polyline as a csv...
csvs_names = Creating_file_names(out_path, "\lines_Split_view", Num_Image_Coords)

edited_csv_list =[]
for csv in csvs_names:
    edited_csv_list.append(csv + ".csv")

for split_lines, csv in zip(Split_line_geom_names, edited_csv_list):
    arcpy.conversion.ExportTable(split_lines, 
                                 csv, 
                                 '', 
                                 "NOT_USE_ALIAS"
                                )

### Scratch Work

In [26]:
from pprint import pprint
# a simply sanity check: length of each dict should eaqual 'Number of Polykines' a.k.a. 501
print(len(max_dict1),len(max_dict2),len(max_dict3),len(max_dict4),len(max_dict5),len(max_dict6))

# pprint(max_dict)

501 501 501 501 501 501
